In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [14]:
import operator
from typing import Annotated, TypedDict, List
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [15]:
tool = TavilySearchResults(max_results=2) 

In [16]:
class AgentState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [18]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, path_map={True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!!!")
        return {"messages": results}

    def exists_action(self, state: AgentState):
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

In [19]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [20]:
messages = [HumanMessage(content="What is the weather in SF?")]

In [21]:
thread = {"configurable": {"thread_id": "1"}}

In [22]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v["messages"])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ehvhX2WGIwI9FXZA7VA7tcBF', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41e844b9-d88e-4cff-bce1-84c0017d1b40-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ehvhX2WGIwI9FXZA7VA7tcBF'}], usage_metadata={'input_tokens': 151, 'output_tokens': 22, 'total_tokens': 173})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ehvhX2WGIwI9FXZA7VA7tcBF'}
Back to the model!!!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Sa

In [24]:
messages = [HumanMessage(content="What about New York?")]
thread = {"configurable": {"thread_id": "1"}}

In [25]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v["messages"])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_v3EpiQ6qBD81GouWQJTQXJQD', 'function': {'arguments': '{"query":"current weather in New York"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 858, 'total_tokens': 880}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-209a4daa-c3fd-4d7b-bd1e-4cdf4fa28895-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New York'}, 'id': 'call_v3EpiQ6qBD81GouWQJTQXJQD'}], usage_metadata={'input_tokens': 858, 'output_tokens': 22, 'total_tokens': 880})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New York'}, 'id': 'call_v3EpiQ6qBD81GouWQJTQXJQD'}
Back to the model!!!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'New York\', \'reg

In [26]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}

In [27]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v["messages"])

[AIMessage(content='Currently, New York is warmer than San Francisco. Here are the temperatures:\n\n- **San Francisco**: 14.0°C (57.2°F)\n- **New York**: 27.2°C (81.0°F)\n\nSo, New York is significantly warmer than San Francisco at the moment.', response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 1547, 'total_tokens': 1611}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-7f79cdc3-26ef-477c-825c-64c1a92920e1-0', usage_metadata={'input_tokens': 1547, 'output_tokens': 64, 'total_tokens': 1611})]


In [28]:
messages = [HumanMessage(content="Which one is warmer?")]
# This wont have any data of previous conversations since the thread id is different
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v["messages"])

[AIMessage(content='Could you please clarify what you are referring to? Are you asking about the weather in two specific locations, the temperature of two objects, or something else?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 149, 'total_tokens': 181}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-323d4f70-4700-4c0a-8dfe-3a0455faf264-0', usage_metadata={'input_tokens': 149, 'output_tokens': 32, 'total_tokens': 181})]


In [30]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:") 
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [34]:
messages = [HumanMessage(content="What's the weather in SF?")]
# This wont have any data of previous conversations since the thread id is different
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events(
    {"messages": messages}, thread, version="v1"
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked
            # So we only print non-empty content
            print(content, end="")
            # print(content, end="|")

The current weather in San Francisco is:
- **Temperature:** 13.0°C (55.4°F)
- **Condition:** Mist
- **Wind:** 6.9 mph (11.2 kph) from WSW
- **Humidity:** 94%
- **Visibility:** 4.0 km (2.0 miles)

![Weather Icon](//cdn.weatherapi.com/weather/64x64/day/143.png)

Is there anything specific you are looking for?